In [ ]:
from bfseg.utils.metrics import IgnorantBalancedMeanIoU, IgnorantMeanIoU, IgnorantBalancedAccuracyMetric, IgnorantAccuracyMetric
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import re
import pandas as pd
from bfseg.data.meshdist.dataLoader import DataLoader



def oneMetricIteration(metric, label, pred):
  """ Helper function to get the result from one prediction """
  metric.update_state(label, pred)
  res = metric.result().numpy()
  metric.reset_states()
  return res


In [ ]:
def evaluate(images, masked = False, assignForeground = False):
    """ evaluates the given pseudo labels"""
    iam_valid = IgnorantAccuracyMetric()
    ibm_valid = IgnorantBalancedAccuracyMetric()
    MIOUM_valid = IgnorantMeanIoU()

    for gt, img in images:
        img_np = np.asarray(Image.open(img))
        pseudo_masked = img_np
        pseudo_labels_unknown_background = img_np > 0
        pseudo_labels_unknown_fg = img_np > 1

        img_np = np.asarray(Image.open(gt).resize((img_np.shape[1], img_np.shape[0]), Image.NEAREST))
        gt_labels = np.sum(img_np, axis = -1) > 0



        gt_labels_ignorant = gt_labels * 2
        if masked:
            gt_labels_ignorant[pseudo_masked == 1] = 1

        # Update Accuracy metrics
        pseudo_labels_cat = tf.keras.utils.to_categorical(
            tf.convert_to_tensor(pseudo_labels_unknown_fg if assignForeground else pseudo_labels_unknown_background), num_classes=3, dtype='float32'
        )

        iam_valid.update_state(gt_labels_ignorant, pseudo_labels_cat)
        ibm_valid.update_state(gt_labels_ignorant, pseudo_labels_cat)
        MIOUM_valid.update_state(gt_labels_ignorant, pseudo_labels_cat)

    iam_valid_value = iam_valid.result().numpy()
    MIOUM_valid_value = MIOUM_valid.result().numpy()
    ibm_valid_value = ibm_valid.result().numpy()

    print("Accuracy on validation set:", iam_valid_value)
    print("Balanced Accuracy on validation set:", ibm_valid_value)
    print("mIoU on validation set:", MIOUM_valid_value)
    capture_values.append((iam_valid_value, ibm_valid_value,MIOUM_valid_value))


In [235]:
DL = DataLoader(
          "/home/rene/cla_dataset/fused", [480, 624],
          validationDir="/home/rene/hiveLabels",
          validationMode="CLA",
          batchSize=1,
          loadDepth=False,
          cropOptions={
              'top': 0,
              'bottom': 0
          })
validLabels = DL.validationLabels

images_fused = []
for label in validLabels:
    # only use images where we have pseudo labels for
    regex_pattern = "(.+)_(\d+\.\d+)_semseg\.png"
    match = re.search(regex_pattern, os.path.basename(label))
    cam = match.group(1)
    ts = float(match.group(2))
    info_file = os.path.join("/home/rene/cla_dataset/fused", f"{cam}_info.txt")
    if(np.sum(pd.read_csv(info_file, header=None, sep=',|;', engine="python").to_numpy()[:,-2] == ts)):
        idx = np.argmax(pd.read_csv(info_file, header=None, sep=',|;', engine="python").to_numpy()[:,-2] == ts)
        img_path = [ l for l in DL.labels if cam + "_img_"+(str(idx).zfill(4)) in l][0]
        images_fused.append((label, img_path))

In [236]:
DL = DataLoader(
          "/home/rene/cla_dataset/sp_slac", [480, 624],
          validationDir="/home/rene/hiveLabels",
          validationMode="CLA",
          batchSize=1,
          loadDepth=False,
          cropOptions={
              'top': 0,
              'bottom': 0
          })
validLabels = DL.validationLabels

images_slac = []
for label in validLabels:
    # only use images where we have pseudo labels for
    regex_pattern = "(.+)_(\d+\.\d+)_semseg\.png"
    #print(os.path.basename(label))
    match = re.search(regex_pattern, os.path.basename(label))
    cam = match.group(1)
    ts = float(match.group(2))
    info_file = os.path.join("/home/rene/cla_dataset/fused", f"{cam}_info.txt")
    if(np.sum(pd.read_csv(info_file, header=None, sep=',|;', engine="python").to_numpy()[:,-2] == ts)):
        idx = np.argmax(pd.read_csv(info_file, header=None, sep=',|;', engine="python").to_numpy()[:,-2] == ts)
        img_path = [ l for l in DL.labels if cam + "_img_"+(str(idx).zfill(4)) in l][0]
        images_slac.append((label, img_path))

In [237]:
DL = DataLoader(
          "/home/rene/cla_dataset/watershed", [480, 624],
          validationDir="/home/rene/hiveLabels",
          validationMode="CLA",
          batchSize=1,
          loadDepth=False,
          cropOptions={
              'top': 0,
              'bottom': 0
          })
validLabels = DL.validationLabels

images_watershed = []
for label in validLabels:
    # only use images where we have pseudo labels for
    regex_pattern = "(.+)_(\d+\.\d+)_semseg\.png"
    match = re.search(regex_pattern, os.path.basename(label))
    cam = match.group(1)
    ts = float(match.group(2))
    info_file = os.path.join("/home/rene/cla_dataset/fused", f"{cam}_info.txt")
    if(np.sum(pd.read_csv(info_file, header=None, sep=',|;', engine="python").to_numpy()[:,-2] == ts)):
        idx = np.argmax(pd.read_csv(info_file, header=None, sep=',|;', engine="python").to_numpy()[:,-2] == ts)
        img_path = [ l for l in DL.labels if cam + "_img_"+(str(idx).zfill(4)) in l][0]
        images_watershed.append((label, img_path))

In [238]:
len(images_watershed)

14

In [246]:
capture_values = []



## Evaluate Labels - Masking out

In [250]:
print("watershed")
evaluate(images_watershed, masked = True)

print("slac")
evaluate(images_slac, masked = True)

print("combined")
evaluate(images_fused, masked = True)

watershed
Accuracy on validation set: 0.8029897
Balanced Accuracy on validation set: 0.7087654
mIoU on validation set: 0.60091454
slac
Accuracy on validation set: 0.84657276
Balanced Accuracy on validation set: 0.7886652
mIoU on validation set: 0.6841648
combined
Accuracy on validation set: 0.8665399
Balanced Accuracy on validation set: 0.78035957
mIoU on validation set: 0.6931578


## Evaluate Labels - Assign BG

In [251]:
print("watershed")
evaluate(images_watershed, masked = False,assignForeground = False)

print("slac")
evaluate(images_slac, masked = False,assignForeground = False)

print("combined")
evaluate(images_fused, masked = False,assignForeground = False)

watershed
Accuracy on validation set: 0.7737981
Balanced Accuracy on validation set: 0.6461298
mIoU on validation set: 0.5300385
slac
Accuracy on validation set: 0.75885826
Balanced Accuracy on validation set: 0.6134369
mIoU on validation set: 0.48906723
combined
Accuracy on validation set: 0.75086814
Balanced Accuracy on validation set: 0.5890446
mIoU on validation set: 0.46131268


## Evaluate Labels - Assign FG

In [252]:
print("watershed")
evaluate(images_watershed, masked = False,assignForeground = True)

print("slac")
evaluate(images_slac, masked = False,assignForeground = True)

print("combined")
evaluate(images_fused, masked = False,assignForeground = True)

watershed
Accuracy on validation set: 0.63819784
Balanced Accuracy on validation set: 0.6389101
mIoU on validation set: 0.4503584
slac
Accuracy on validation set: 0.51139015
Balanced Accuracy on validation set: 0.60945547
mIoU on validation set: 0.34353313
combined
Accuracy on validation set: 0.5115174
Balanced Accuracy on validation set: 0.61108446
mIoU on validation set: 0.3436405


In [254]:
capture_values # watershed, slac, combined

[(0.8029897, 0.7087654, 0.60091454),
 (0.84657276, 0.7886652, 0.6841648),
 (0.8665399, 0.78035957, 0.6931578),
 (0.7737981, 0.6461298, 0.5300385),
 (0.75885826, 0.6134369, 0.48906723),
 (0.75086814, 0.5890446, 0.46131268),
 (0.63819784, 0.6389101, 0.4503584),
 (0.51139015, 0.60945547, 0.34353313),
 (0.5115174, 0.61108446, 0.3436405)]

In [259]:
[(round(v[0]*100,1),round(v[1]*100,1),round(v[2]*100,1)) for v in capture_values]

[(80.3, 70.9, 60.1),
 (84.7, 78.9, 68.4),
 (86.7, 78.0, 69.3),
 (77.4, 64.6, 53.0),
 (75.9, 61.3, 48.9),
 (75.1, 58.9, 46.1),
 (63.8, 63.9, 45.0),
 (51.1, 60.9, 34.4),
 (51.2, 61.1, 34.4)]